<font size="6"><b>Incentivized Reviews Algorithm</b></font>

This notebook serves to shortlist a list of reviews that have a high probability of being incentivized based on a 2-step process of text analysis combined with a classifier model built on a dataset containing over 150 incentivized reviews.

In order to analyze future ASIN's, the following files are required:<br>
- This notebook
- Competitor_incentivized_model.sav 
- useful_functions.py
- RSC_Competitor_Reviews.csv (Dataset used to build the classifier)

In [0]:
# We begin by importing the useful functions used for the first step of the process
from useful_functions import *

# Step 1: Load Data, attach text features

In [0]:
# Option shall be set to 1 for any new dataset 
option=1

############################################ INPUT FILE NAME HERE ############################################

datafile = "Products_for_Incentivized_Reviews_Test.csv"

# This option is used to run the dataset used to train the classifier model
if option==0:
    df= pd.read_csv('RSC_Competitor_Reviews.csv')
    print("Let's test run, we have built-in manually labeled data")
else:
    df=pd.read_csv(datafile)


df=df.dropna(axis=1,thresh=len(df)*0.9)
df.head()

,source,product,name,date,author,profile,verified,stars,pvotes,tvotes,title,text,image,video,reviewid,reviewlink,parent,inputtime,product_retailer,product_id
0,amazon.ca,B00LM2Y2U4,UGREEN 3.5mm to 2 RCA Male to Male Aux Audio C...,4/28/20 8:00,Dewey T.,https://www.amazon.ca/gp/profile/amzn1.account...,True,5,0,0,"Good quality, good price.",Exactly what I needed.,No,No,R3BXKP401TQ7NG,https://www.amazon.ca/gp/customer-reviews/R3BX...,B085S4QYBB,4/28/20 8:00,amazon.ca,B00LM2Y2U4
1,amazon.ca,B00LM2Y2U4,UGREEN 3.5mm to 2 RCA Male to Male Aux Audio C...,4/22/20 8:00,Nick,https://www.amazon.ca/gp/profile/amzn1.account...,True,5,0,0,Solid Cable!,"Cell phone to stereo. Perfect, thin 3.5mm end ...",No,No,R3LL1VGOUITKN,https://www.amazon.ca/gp/customer-reviews/R3LL...,B085S4QYBB,4/22/20 8:00,amazon.ca,B00LM2Y2U4
2,amazon.ca,B00LM2Y2U4,UGREEN 3.5mm to 2 RCA Male to Male Aux Audio C...,4/20/20 8:00,Pierre,https://www.amazon.ca/gp/profile/amzn1.account...,True,5,0,0,S'adapte parfaitement même si mon équipement à...,Très bonne qualité et bon prix,No,No,R25NB94MV0R6D3,https://www.amazon.ca/gp/customer-reviews/R25N...,B085S4QYBB,4/20/20 8:00,amazon.ca,B00LM2Y2U4
3,amazon.ca,B00LM2Y2U4,UGREEN 3.5mm to 2 RCA Male to Male Aux Audio C...,4/9/20 8:00,Amazon Customer,https://www.amazon.ca/gp/profile/amzn1.account...,True,5,0,0,Good Value,My old cord struggled to fit though my ipad an...,No,No,R2IABXJYU7A01L,https://www.amazon.ca/gp/customer-reviews/R2IA...,B085S4QYBB,4/9/20 8:00,amazon.ca,B00LM2Y2U4
4,amazon.ca,B00LM2Y2U4,UGREEN 3.5mm to 2 RCA Male to Male Aux Audio C...,3/9/20 8:00,Tyler,https://www.amazon.ca/gp/profile/amzn1.account...,True,5,0,0,Good,Does what it's supposed to,No,No,R37KYIJ8C1MH3I,https://www.amazon.ca/gp/customer-reviews/R37K...,B085S4QYBB,3/9/20 8:00,amazon.ca,B00LM2Y2U4


In [0]:
# Creating a list of suspicious words associated with incentivized behavior
def create_incentivized_words():
    '''
       input: a list shows incentivized activity
       output: deduplicated lists in clean text format
    
       sample: incentivized_list=create_incentivized_words()

    '''
    incentivized_words=[
    "Free one"
    ,"Free product"
    ,"for a positive review"
    ,"Free gift"
    ,"Promised a free"
    ,"Another free"
    ,"In exchange for a positive review"
    ,"Ask positive review"
    ,"In exchange for a review"
    ,"If you review"
    ,"If I reviewed the product"
    ,"Write a review"
    ,"Writing a review"
    ,"Leave us a review"
    ,"Leave a review"
    ,"Glowing review"
    ,"Positive review"
    ,"If I Left a review"
    ,"Reviews are paid"
    ,"Review in return"
    ,"For a review"
    ,"For our review"
    ,"For my review"
    ,"Leave a 5 star review"
    ,'5 star review'
    ,"Incentive"
    ,"Incentivized"
    ,"Gift card"
    ,"Inside the packaging was a flyer"
    ,"Flyer"
    ,"Bribe"
    ,"A favorable review"
    ,"In return for a review"
    ,"In return for a good review"
    ,"Star amazon review"
    ,"Gift for this review"
    ,"In exchange for writing"]
    
    normalize_corpus=np.vectorize(normalize_document)
    cleaned= normalize_corpus(incentivized_words)
    cleaned=cleaned[cleaned!='review']

    incentivized_words_list=[]
    [incentivized_words_list.append(x) for x in cleaned if x not in incentivized_words_list]
    print(incentivized_words_list)
    return(incentivized_words_list)

In [0]:
# Generate Text Features
texts=df['text'].apply(str)
text_features=generate_text_features(texts)
df=pd.concat([df,text_features],axis=1)

normalize_corpus=np.vectorize(normalize_document)
texts_clean= normalize_corpus(texts)


# Get a list of incentivized words
incentivized_words_cleaned=create_incentivized_words()

# Initial Incentivized Label
vector=[]
for text in texts_clean:
    if any(word in text for word in incentivized_words_cleaned):
        vector.append(1)
    else:
        vector.append(0)


# Add clean text column to original dataframe
texts_clean_list = texts_clean.tolist()
df['clean_text'] = texts_clean_list



['free one', 'free product', 'positive review', 'free gift', 'promise free', 'another free', 'exchange positive review', 'ask positive review', 'exchange review', 'review product', 'write review', 'leave us review', 'leave review', 'glow review', 'review pay', 'review return', 'leave star review', 'star review', 'incentive', 'incentivized', 'gift card', 'inside package flyer', 'flyer', 'bribe', 'favorable review', 'return review', 'return good review', 'star amazon review', 'gift review', 'exchange write']


In [0]:
print("Total incentivized reviews= {} ".format(sum(vector)))

print("{} percent of the reviews may be incentivized ".format(round(sum(vector)/df.shape[0]*100),2))
df['incentivized_reviews']=vector


Total incentivized reviews= 42 
1 percent of the reviews may be incentivized 


# Step 2: Manual Labelling of our Classifier Model

In [0]:
# Let's take columns of interest
output = df[['reviewid','incentivized_reviews','text','clean_text','text_polarity']]

In [0]:
# Let us look only at reviews flagged as incentivized
refined_output = output[output['incentivized_reviews']==1]
refined_output.head(3)

,reviewid,incentivized_reviews,text,clean_text,text_polarity
1464,R3CKLK2WHOHCB2,1,So I definitely should have written this revie...,definitely write review like year ago promise ...,0.14793
1581,R3CVH7P6M095LW,1,This bed cover was purchased in conjunction wi...,bed cover purchase conjunction new mattress la...,0.07697
1791,R2TVN1OU1WJKYO,1,I don't understand why ppl wrote reviews sayin...,dont understand ppl write review say cant tell...,0.20000


In [0]:
if(option==0):
    Manual_Labels = [
           0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
           0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1,
           0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0,
           1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1,
           0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
           0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
           0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
           1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0,
           1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
           1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
           0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0,
           0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
           0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1,
           1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,
           1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1,
           1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
           1, 0, 0, 1]
    # Adding Manual Labels to our dataframe of interest
    refined_output['Manual_Label'] = Manual_Labels

# Step 3: Classification Preparation

## Creating flags for specific words

In [0]:
# Add few words along to incentivized list
new_word_filter = incentivized_words_cleaned + ['disclosure','gift','card','free','slip']

In [0]:
# Check if new word filter exists in clean text
for item in new_word_filter:
    refined_output[item] = refined_output['clean_text'].str.lower().str.contains(item)
refined_output

/Users/kkhaled/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,reviewid,incentivized_reviews,text,clean_text,text_polarity,free one,free product,positive review,free gift,promise free,...,return review,return good review,star amazon review,gift review,exchange write,disclosure,gift,card,free,slip
1464,R3CKLK2WHOHCB2,1,So I definitely should have written this revie...,definitely write review like year ago promise ...,0.147930,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1581,R3CVH7P6M095LW,1,This bed cover was purchased in conjunction wi...,bed cover purchase conjunction new mattress la...,0.076970,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1791,R2TVN1OU1WJKYO,1,I don't understand why ppl wrote reviews sayin...,dont understand ppl write review say cant tell...,0.200000,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1915,R12T8C9RIOHGN9,1,Such a small price to pay for the safety of yo...,small price pay safety mattress zipper cover p...,0.184607,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2264,R20WSSANRFXJBL,1,I was not going to write a review because I wa...,go write review please product receive itit ea...,0.129252,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2593,R1BOYR8NU4ERST,1,I was excited to receive this product because ...,excite receive product favorable review howeve...,0.125000,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3210,RI236QUGVH91S,1,DONT BE MISLED BY THOSE POSITIVE REVIEWS. The ...,dont mislead positive review buyers must unawa...,0.061039,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3717,R30SQ4A3U3K999,1,"Works as expected, a little lightweight, but t...",work expect little lightweight might good thin...,0.118403,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3968,R2OK2870VLG4IV,1,I wish I'd paid attention to the one star revi...,wish id pay attention one star review buy put ...,0.066667,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4271,R2KTK849YX9FTA,1,I have had this thing for a year or two and to...,thing year two today stop work try multiple co...,0.171605,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [0]:
# change boolean to integers 0/1
refined_output.iloc[:,5:] = refined_output.iloc[:,5:].astype(int)
refined_output.describe()

/Users/kkhaled/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,incentivized_reviews,text_polarity,free one,free product,positive review,free gift,promise free,another free,exchange positive review,ask positive review,...,return review,return good review,star amazon review,gift review,exchange write,disclosure,gift,card,free,slip
count,42.0,42.000000,42.0,42.0,42.000000,42.000000,42.000000,42.0,42.0,42.0,...,42.0,42.0,42.0,42.0,42.0,42.0,42.000000,42.000000,42.000000,42.0
mean,1.0,0.228335,0.0,0.0,0.071429,0.333333,0.023810,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.357143,0.119048,0.380952,0.0
std,0.0,0.138261,0.0,0.0,0.260661,0.477119,0.154303,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.484966,0.327770,0.491507,0.0
min,1.0,-0.097109,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
25%,1.0,0.132197,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
50%,1.0,0.192304,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
75%,1.0,0.342976,0.0,0.0,0.000000,1.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,1.000000,0.0
max,1.0,0.497143,0.0,0.0,1.000000,1.000000,1.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000,0.0


In [0]:
# Let's take a look
refined_output.head(3)

,reviewid,incentivized_reviews,text,clean_text,text_polarity,free one,free product,positive review,free gift,promise free,...,return review,return good review,star amazon review,gift review,exchange write,disclosure,gift,card,free,slip
1464,R3CKLK2WHOHCB2,1,So I definitely should have written this revie...,definitely write review like year ago promise ...,0.14793,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1581,R3CVH7P6M095LW,1,This bed cover was purchased in conjunction wi...,bed cover purchase conjunction new mattress la...,0.07697,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1791,R2TVN1OU1WJKYO,1,I don't understand why ppl wrote reviews sayin...,dont understand ppl write review say cant tell...,0.20000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [0]:
# Reset Index
classifier_data = refined_output.reset_index(drop=True)

## Check for combination of incentivized list with the new words

In [0]:
# For the ['disclosure','gift','card','free','review pay','slip'], We'd just like them to be picked up in combination with other words
def manual_FE(classifier_data,option):
    start_num,end_num=6,35
    col=36
    if(option==1):
        start_num-=1
        end_num-=1
        col-=1
    
    classifier_data['disclosure_combo'] = classifier_data.iloc[:,np.r_[start_num:end_num, col]].sum(axis=1) - classifier_data.iloc[:,start_num:end_num].sum(axis=1)
    classifier_data['gift_combo'] = classifier_data.iloc[:,np.r_[start_num:end_num, col+1]].sum(axis=1) - classifier_data.iloc[:,start_num:end_num].sum(axis=1)
    classifier_data['card_combo'] = classifier_data.iloc[:,np.r_[start_num:end_num, col+2]].sum(axis=1) - classifier_data.iloc[:,start_num:end_num].sum(axis=1)
    classifier_data['free_combo'] = classifier_data.iloc[:,np.r_[start_num:end_num, col+3]].sum(axis=1) - classifier_data.iloc[:,start_num:end_num].sum(axis=1)
    classifier_data['slip_combo'] = classifier_data.iloc[:,np.r_[start_num:end_num, col+4]].sum(axis=1) - classifier_data.iloc[:,start_num:end_num].sum(axis=1)

    # Create flag if any of the combos are returned
    classifier_data['any_combo'] = classifier_data.iloc[:,col+5:col+10].max(axis=1)

    # What if picking up more than 1 review is important?
    # Let's create a new column to see the number of features flagged
    classifier_data['flags'] = classifier_data.iloc[:,np.r_[start_num:end_num, col+10]].sum(axis=1)
    return(classifier_data['flags'])

In [0]:
classifier_data['flags']=manual_FE(classifier_data,option)

## Create flags for the number of flags and rows that picked up more than 1 flag

In [0]:
# Create a new column if flags > 1 
classifier_data['flags_greater_than_1'] = classifier_data['flags'].apply(lambda x: 1 if x > 1 else 0)

In [0]:
classifier_data.head()

,reviewid,incentivized_reviews,text,clean_text,text_polarity,free one,free product,positive review,free gift,promise free,...,free,slip,disclosure_combo,gift_combo,card_combo,free_combo,slip_combo,any_combo,flags,flags_greater_than_1
0,R3CKLK2WHOHCB2,1,So I definitely should have written this revie...,definitely write review like year ago promise ...,0.147930,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,R3CVH7P6M095LW,1,This bed cover was purchased in conjunction wi...,bed cover purchase conjunction new mattress la...,0.076970,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,R2TVN1OU1WJKYO,1,I don't understand why ppl wrote reviews sayin...,dont understand ppl write review say cant tell...,0.200000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,R12T8C9RIOHGN9,1,Such a small price to pay for the safety of yo...,small price pay safety mattress zipper cover p...,0.184607,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,R20WSSANRFXJBL,1,I was not going to write a review because I wa...,go write review please product receive itit ea...,0.129252,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


# Step 4: Building a Random Forest Classifier

In [0]:
# Import Packages
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
import statsmodels.api as sm
from sklearn import svm
from sklearn.metrics import roc_curve, precision_recall_curve, auc 
from sklearn.metrics import make_scorer, recall_score, accuracy_score, precision_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [0]:
n=6
if(option!=0):
    n=n-1
x = classifier_data.iloc[:,n:]
# drop unneccessary columns
x.drop(['return review'], axis=1, inplace = True)
x.drop(['leave us review'], axis=1, inplace = True)
x.drop(['flyer'], axis=1, inplace = True)
x.drop(['inside package flyer'], axis=1, inplace = True)
x.drop(['incentivized'], axis=1, inplace = True)
x.drop(['exchange positive review'], axis=1, inplace = True)
x.drop(['bribe'], axis=1, inplace = True)
x.drop(['disclosure'], axis=1, inplace = True)
x.drop(['gift'], axis=1, inplace = True)
x.drop(['card'], axis=1, inplace = True)
x.drop(['free'], axis=1, inplace = True)
x.drop(['slip'], axis=1, inplace = True)
x.drop(['disclosure_combo'], axis=1, inplace = True)
x.drop(['gift_combo'], axis=1, inplace = True)
x.drop(['card_combo'], axis=1, inplace = True)
x.drop(['free_combo'], axis=1, inplace = True)
x.drop(['slip_combo'], axis=1, inplace = True)

In [0]:
import pickle
if(option==0):
    y = classifier_data['Manual_Label']
    
    # Split into training and testing data with 70% training 30% testing
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 1, train_size = 0.7)
    
    random_forest = RandomForestClassifier()
    random_forest.fit(x_train, y_train) #model fitting
    print('Accuracy of random forest classifier on test set: {:.2f}'.format(random_forest.score(x_test, y_test)))
    
    
    y_pred = random_forest.predict(x_test)
    # Confusion matrix
    confusion_matrix_random_forest = confusion_matrix(y_test, y_pred)
    print(confusion_matrix_random_forest)
    print ('\n')
    print ("=== Classification Report ===")
    print (classification_report(y_test, y_pred))
    print ('\n')
    
    # Generate the model
    filename = 'Competitor_incentivized_model.sav'
    pickle.dump(random_forest, open(filename, 'wb'))
    
else:
    filename = 'Competitor_incentivized_model.sav'
    random_forest = pickle.load(open(filename, 'rb'))
    # Predict on the previously trained model
    y_pred = random_forest.predict(x)
    print(y_pred)

[0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 1 0 1 1 1 0
 1 0 1 0 0]


In [0]:
# This is to check the performance of the classifier for the trained dataset (option 0)
if(option==0):
    # Feature importance
    feat_labels = x_train.columns
    importances = random_forest.feature_importances_
    indices = np.argsort(importances)[::-1]
    for f in range(x_train.shape[1]):
        print("%2d) %-*s %f" % (f + 1, 30, feat_labels[indices[f]], importances[indices[f]]))

# Step 5: Providing the list of Incentivized Reviews

In [0]:
if(option!=0):
    Filtered_data = classifier_data[['reviewid','text']]
    Filtered_data['Incentivized'] = y_pred
    Predicted_data = Filtered_data[Filtered_data['Incentivized'] == 1]
    Predicted_data.reset_index(inplace = True, drop=True)
    Predicted_data.drop(['Incentivized'],axis=1,inplace = True)
    Predicted_data.to_csv("predicted.csv")

/Users/kkhaled/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/kkhaled/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
Predicted_data

,reviewid,text
0,R2OK2870VLG4IV,I wish I'd paid attention to the one star revi...
1,R2KTK849YX9FTA,I have had this thing for a year or two and to...
2,R3N47UBSH0UWQ,**EDIT** (Original Review Below):After posting...
3,R1OHU7F7ZYYCII,Favorable reviews I saw before purchasing this...
4,R230YCC5DAK05E,My (very) old SD flash drive no longer worked ...
5,R1LV96DHSZX6W1,They told me to give them a 5-star review and ...
6,R3LN11S4KXET72,What can I say that has not already been state...
7,R2L4MHL87GXQMC,"Purchased this projector, 100inch screen, and ..."
8,R118STA1HLTL8E,I got these projectors for my church. We have...
9,R36EPJR5PAQY0K,We use this for mainly movies and it fits our ...
